In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML

# Diskretna Fourierova transformacija

Diskretna Fourierova transformacija (DFT) nam nudi matematično orodje za dekompozicijo originalnega signala na sinusoide. Zanima nas torej s katerimi sinusi in kosinusi lahko predstavimo signal. Signal označimo z x, medtem ko bomo njegovo transformiranko označili z X. Signal razstavljamo na sinusoide, saj sinus pri prehodu skozi sistem ne spremeni svoje frekvence temveč le amplitudo in fazo (zamik). Tako dobimo idealno orodje za opazovanje dogajanja pri posamezni frekvenci.


## Razlika med časovnim in frekvenčnim prostorom

In [ ]:
Fvz = 150  # vzorčevalna frekvenca
T = 3 # dolžina signala v sekundah
t = np.arange(0, T, 1.0/Fvz) # časovni vektor

f = 5   # # frkevenca sinusoide

y = np.sin(2 * np.pi * f * t)

n = len(y)
k = np.arange(n)

frq = k/T # celotno frekvenčno območje
#frq = frq[np.arange(int(n/2))] # polovično frekvenčno območje

Y = np.fft.fft(y)/n # izračun fft in normalizacija
#Y = Y[np.arange(int(n/2))] # polovično frekvenčno območje

fig = plt.figure()

plt.subplot(2, 1, 1)

plt.plot(t, y)

plt.title('Časovni prostor')
plt.xlabel('čas (s)')
plt.ylabel('amplituda')

plt.subplot(2, 1, 2)

plt.plot(frq, abs(Y)) 

plt.title('Frekvenčni prostor')
plt.xlabel('Frekvenca (Hz)')
plt.ylabel('amplituda')

plt.tight_layout()

plt.show()
fig


## Zakaj ravno dekompozicija na sinusoide?

Če je vhod v linearni sistem sinusoida, bo tudi izhod iz sistema sinusoida z enako frekvenco. Spremeni se lahko le amplituda in faza. Sinusoide so edini signali s to lastnostjo. Z dekompozicijo signal na posamezne sinusoide dobimo torej močno orodje za opazovanje, kaj se dogaja s posamezno frekvenco v sistemu.

Poudarimo da je pri DFT signal diskreten in neskončen! V računalništvu neskončen signal ne obstaja, zato predpostavimo, da je signal periodičen in da imamo v pomnilniku le eno celo periodo.

Pripravimo si preprost sistem.

In [ ]:
def sistem(x):
    h = np.array([1, -1])
    y = np.convolve(x, h, mode='same')
   
    return y

Pripravimo trikotni signal.

In [ ]:
x = np.concatenate((np.linspace(0, 1, 10, endpoint=False), 
                    np.linspace(1, -1, 20, endpoint=False), 
                    np.linspace(-1, 0, 10, endpoint=False)))
x = np.tile(x, 5)

In [ ]:
fig = plt.figure()
plt.plot(x, 'o-')
plt.title('x - trikotni signal')
plt.xlabel('vzorec')
plt.ylabel('amplituda')
fig


Podajmo ta trikotni signal skozi naš sistem in poglejmo kakšen bo izhod.



In [ ]:
y=sistem(x)

# risanje

In [ ]:
fig = plt.figure()

plt.plot(y, 'o-')
plt.title('y - output')
plt.xlabel('vzorec')
plt.ylabel('amplituda')

plt.tight_layout()
fig

V primeru je vhodni signal bil trikoten, izhod sistema pa je bil pravokoten signal. Frekvenca signala je sicer ostala ista, bistveno se je spremenila pa tudi amplituda signala.

Če boste bolj natančno preučite sistem, in primerjali vhod in izhod boste morda ugotovili kaj izhod našega sistema dejansko predstavlja. Sistem simulira zelo pomembno operacijo.

Kaj se zgodi, če na vhod sistema pošljemo sinusoido? Preizkusimo nekaj različnih primerov.

In [ ]:
Fvz=50
f_list = [1, 2, 4] # frekvence, ki jih bomo preizkusili

fig, ax_table = plt.subplots(2, len(f_list), sharex=True, sharey=True)

for ax in ax_table[:, 0]:
    ax.set_ylabel('amplituda')

for ax in ax_table[1, :]:
    ax.set_xlabel('vzorec')

# za vsako frekvenco
for f, ax_in, ax_out in zip(f_list, ax_table[0, :], ax_table[1, :]):    
    
    # pripravimo sinus izbrane frekvence
    x = np.sin(np.arange(0, Fvz)*2*np.pi*f/Fvz)
    # in ga posljimo skozi sistem
    y=sistem(x)

    # risanje
    ax_in.set_title("x - input")
    ax_in.plot(x)
    ax_out.set_title('y - output')
    ax_out.plot(y)
plt.tight_layout()
fig

V prvi vrstici imamo tri primere sinusoid, s frekvenco 1, 2 in 4 Hz.

V drugi vrstici imamo izhode sistema za pripravljene sinusoide. Tudi tukaj vidimo 3 sinusoide s frekvencami 1, 2 in 4 Hz. Opazimo pa tudi, da sta se amplituda in zamik teh sinusoid (faza) spremenila.

V izhodih vidimo tudi, da je prva vrednost 0. To je posledica prehodnega pojava pri računanju konvolucije (na prejšnjih vajah smo si pogledali, kaj naredimo ko del impulznega odziva pade izven mej signala). Gre torej za artefakt naše diskretne implementacije.

**Ker linearni sistemi ne spremenijo oblike sinusoide ampak samo njihovo amplitudo in fazo, so le te idealne za analizo signalov in sistemov.** Če lahko signal predstavimo kot vsoto sinusoid, moramo samo vedeti kako opazovan sistem spremeni posamezno sinusoido in bomo vedeli kako bo sistem vplival na signal.

## Skalarni produkt dveh sinusoid

Frekvenca sinusa pove, koliko nihajev opravi v sekundi. Pri DFT signal razstavljamo na sinusoide s točno določeno frekvenco. Tem sinusoidam pravimo tudi bazne funkcije. Bazne funkcije generiramo z naslednjima enačbama:

$$
c_k[n] = cos\left( \frac{2 \pi kn}{N}\right)
$$
$$ 
s_k[n] = sin\left( \frac{2 \pi kn}{N}\right)
$$

Kjer je N dolžina signala, ki ga transformiramo, n indeks ki gre od 0 do N-1, ter k indeks, ki pove frekvenco sinusa oz. kosinusa. Frekvenca k mora biti obvezno celo število med [0, N/2].

Povejmo, da so bazne funkcije med seboj ortogonalne (pravokotne). To pomeni, da je skalarni produktov dveh baznih funkcij z različno frekvenco enaka 0.

In [ ]:
N = 100
n_list = np.arange(N) # vsi indeksi n

# bazne funkcije s k=3
k = 3
sin_3 = np.sin(2*np.pi* k*n_list/N)
cos_3 = np.cos(2*np.pi* k*n_list/N)

# bazne funkcije s k=4
k = 4
sin_4 = np.sin(2*np.pi*k*n_list/N)
cos_4 = np.cos(2*np.pi*k*n_list/N)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)

ax[0, 0].plot(sin_3)
ax[0, 1].plot(sin_4)
ax[1, 0].plot(cos_3)
ax[1, 1].plot(cos_4)

ax[0, 0].set_title('sin_3')
ax[0, 1].set_title('sin_4')
ax[1, 0].set_title('cos_3')
ax[1, 1].set_title('cos_4')

ax[0, 0].set_ylabel('amplituda')
ax[1, 0].set_ylabel('amplituda')
ax[1, 0].set_xlabel('vzorec')
ax[1, 1].set_xlabel('vzorec')

In [ ]:
v1 = sin_3.dot(cos_3)
v2 = sin_3.dot(sin_4)
v3 = sin_3.dot(cos_4)
v4 = sin_3.dot(sin_3)

print(f'sin_3.dot(cos_3) == {v1}')
print(f'sin_3.dot(sin_4) == {v2}')
print(f'sin_3.dot(cos_4) == {v3}')
print(f'sin_3.dot(sin_3) == {v4}')

## Skalarni produkt dveh RAZLIČNIH sinusoid pri RAZLIČNIH fazah


In [ ]:
%%capture 
Fvz = 512 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
i = np.arange(0, T*Fvz) /Fvz # casovni trenutki
f1 = 5 # frkevenca prve sinusoide v Hz
f2 = 6 # frekvenca druge sinusoide v Hz - (spremeni v 5, kaj se zgodi?)

s1 = np.sin(2 * np.pi * f1 * i)
s2 = np.sin(2 * np.pi * f2 * i)

faze_list = np.arange(-1, 1, 0.05)
skalarni_produkt = np.zeros(faze_list.shape[0])

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.plot(i, s1)
plt.title(f'{f1} Hz')
s2_line, = plt.plot(i, s2)
ax1_title = plt.title(f'{f2} Hz, faza = {0} pi')

plt.subplot(2, 1, 2)
sk_prod_line, = plt.plot(faze_list*np.pi, skalarni_produkt)
sk_prod_dot, = plt.plot(faze_list[0]*np.pi, skalarni_produkt[0], 'ro')
plt.xlabel('faza s2')
plt.ylabel('s1.dot(s2)')
plt.tight_layout()

def animate(n):
    
    faza = faze_list[n]
    s2 = np.sin(2 * np.pi * f2 * i + faza * np.pi)
    skalarni_produkt[n] = s1.dot(s2)
    
    s2_line.set_data(i, s2)
    ax1_title.set_text(f's1 = {f1} Hz, s2={f2} Hz, faza {faza:.2f} pi')
    
    sk_prod_line.set_data(faze_list*np.pi, skalarni_produkt)
    sk_prod_dot.set_data(faze_list[n]*np.pi, skalarni_produkt[n])
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(faze_list.shape[0]),
                              interval=50,
                              blit=False,
                              repeat=False)

In [ ]:
HTML(anim.to_jshtml())

## Skalarni produkt dveh sinusoid z ENAKIMA frekvencama in RAZLIČNIMA fazama

In [ ]:
%%capture

Fvz = 512 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
i = np.arange(0, T*Fvz) / Fvz # casovni trenutki
f1 = 5 # frkevenca prve sinusoide v Hz
f2 = 5 # frekvenca druge sinusoide v Hz

A = 1 # amplituda

faza1 = 0.0
faza2 = 1.0

s1 = A * np.sin(2 * np.pi * f1 * i + faza1 * np.pi)
s2 = A * np.sin(2 * np.pi * f2 * i + faza2 * np.pi)


faze_list = np.arange(-1, 1, 0.05)
skalarni_produkt = np.zeros(faze_list.shape[0])

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
plt.plot(i, s1)
plt.title(f'{f1} Hz')
ax1_s2_line, = plt.plot(i, s2)
ax1_title = plt.title(f'{f2} Hz, faza = {0} pi')

ax2 = plt.subplot(2, 1, 2)
plt.xlabel('Faza s2 (Hz)')
sk_prod_line, = plt.plot(faze_list, skalarni_produkt)
sk_prod_dot,  = plt.plot(faze_list[0], skalarni_produkt[0], 'or')
ax2.set_ylim(-256, 256)
plt.tight_layout()

def animate(n):
    faza2 = faze_list[n]
    s2 = A * np.sin(2 * np.pi * f2 * i + faza2 * np.pi)
    skalarni_produkt[n] = s1.dot(s2)
    
    # risanje
    ax1_s2_line.set_data(i, s2)
    ax1_title.set_text(f's1 = {f1} Hz, s2={f2} Hz, razlika faze {faza2:.2f} pi')
    
    sk_prod_line.set_data(faze_list, skalarni_produkt)
    sk_prod_dot.set_data(faze_list[n], skalarni_produkt[n])
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(faze_list.shape[0]),
                              interval=50,
                              blit=False)

In [ ]:
HTML(anim.to_jshtml())

## Analiza s sinusoidami in kosinusoidami

### Spreminjanje faze

In [ ]:
%%capture

Fvz = 512 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
n = np.arange(0, T*Fvz) # casovni trenutki

N = int(Fvz*T)
t = n/Fvz

f = 3 # frkevenca sinusoide v Hz
A = 1 # amplituda
faza = 0

s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal

# k == 3 
sin_3 = np.sin(2 * np.pi  * n*3/N) # sinusoida
cos_3 = np.cos(2 * np.pi  * n*3/N) # kosinusoida


sin_s1 = sin_3.dot(s1.T)
cos_s1 = cos_3.dot(s1.T)

fig = plt.figure()
plt.subplot(3, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
ax1_s1_line, = plt.plot(t, s1)
ax1_title = plt.title(f'{f1} Hz, faza = {0} pi')


plt.subplot(3, 1, 2)
plt.title(f'{f1} Hz')
plt.plot(t, sin_3, label='sin')
plt.plot(t, cos_3, label='cos')
plt.legend()
plt.title('bazni funkciji sin in cos')


ax2 = plt.subplot(3, 1, 3)
ax2_sk_prod_line, = plt.plot([0, cos_s1], [0, sin_s1], '.-')
ax2.set_xlim(-600, 600)
ax2.set_ylim(-600, 600)
ax2.set_aspect('equal')
plt.xlabel('x os')
plt.ylabel('y os')
plt.tight_layout()
    


faze_list = np.arange(-1, 1, 0.05)
def animate(i):
    # pripravimo nas signal
    faza = faze_list[i]
    s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal
    
    # mnozimo z baznimi funkcijami
    sin_s1 = sin_3.dot(s1.T)
    cos_s1 = cos_3.dot(s1.T)
    
    # izris
    ax1_s1_line.set_data(t, s1)
    ax1_title.set_text(f's1 = {f} Hz, amplituda {A:.2f}, faza {faza:.2f} pi')
    
    ax2_sk_prod_line.set_data([0, cos_s1], [0, sin_s1])
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(faze_list.shape[0]),
                              interval=50,
                              blit=False)


In [ ]:
HTML(anim.to_jshtml())

### Spreminjanje amplitude

In [ ]:
%%capture

Fvz = 512 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
n = np.arange(0, T*Fvz) # casovni trenutki

N = int(Fvz*T)
t = n/Fvz

f = 3 # frkevenca sinusoide v Hz
A = 2 # amplituda
faza = 0.25

s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal

# k == 3 
sin_3 = np.sin(2 * np.pi  * n*3/N) # sinusoida
cos_3 = np.cos(2 * np.pi  * n*3/N) # kosinusoida


sin_s1 = sin_3.dot(s1.T)
cos_s1 = cos_3.dot(s1.T)

fig = plt.figure()
plt.subplot(3, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
ax1_s1_line, = plt.plot(t, s1)
ax1_title = plt.title(f'{f1} Hz, faza = {0} pi')


plt.subplot(3, 1, 2)
plt.title(f'{f1} Hz')
plt.plot(t, sin_3, label='sin')
plt.plot(t, cos_3, label='cos')
plt.legend()
plt.title('bazni funkciji sin in cos')


ax2 = plt.subplot(3, 1, 3)
ax2_sk_prod_line, = plt.plot([0, cos_s1], [0, sin_s1], '.-')
ax2.set_xlim(-600, 600)
ax2.set_ylim(-600, 600)
ax2.set_aspect('equal')
plt.xlabel('x os')
plt.ylabel('y os')
plt.tight_layout()

amplitude_list = np.arange(-2, 2, 0.2)

def animate(i):
    # pripravimo nas signal
    A = amplitude_list[i]
    s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal
    
    # mnozimo z baznimi funkcijami
    sin_s1 = sin_3.dot(s1.T)
    cos_s1 = cos_3.dot(s1.T)
    
    # izris
    ax1_s1_line.set_data(t, s1)
    ax1_title.set_text(f's1 = {f} Hz, amplituda {A:.2f}, faza {faza:.2f} pi')
    
    ax2_sk_prod_line.set_data([0, cos_s1], [0, sin_s1])
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(amplitude_list.shape[0]),
                              interval=50,
                              blit=False)

In [ ]:
HTML(anim.to_jshtml())

### Spreminjanje frekvence

In [ ]:
%%capture

Fvz = 512 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
n = np.arange(0, T*Fvz) # casovni trenutki

N = int(Fvz*T)
t = n/Fvz

f = 3 # frkevenca sinusoide v Hz
A = 2 # amplituda
faza = 0.25

s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal

# k == 3 
sin_3 = np.sin(2 * np.pi  * n*3/N) # sinusoida
cos_3 = np.cos(2 * np.pi  * n*3/N) # kosinusoida


sin_s1 = sin_3.dot(s1.T)
cos_s1 = cos_3.dot(s1.T)

fig = plt.figure()
plt.subplot(3, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
ax1_s1_line, = plt.plot(t, s1)
ax1_title = plt.title(f'{f1} Hz, faza = {0} pi')


plt.subplot(3, 1, 2)
plt.title(f'{f1} Hz')
plt.plot(t, sin_3, label='sin')
plt.plot(t, cos_3, label='cos')
plt.legend()
plt.title('bazni funkciji sin in cos')


ax2 = plt.subplot(3, 1, 3)
ax2_sk_prod_line, = plt.plot([0, cos_s1], [0, sin_s1], '.-')
ax2.set_xlim(-600, 600)
ax2.set_ylim(-600, 600)
ax2.set_aspect('equal')
plt.xlabel('x os')
plt.ylabel('y os')
plt.tight_layout()

frekvence_list = np.arange(0, 10, 1)

def animate(i):
    # pripravimo nas signal
    f = frekvence_list[i]
    s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal
    
    # mnozimo z baznimi funkcijami
    sin_s1 = sin_3.dot(s1.T)
    cos_s1 = cos_3.dot(s1.T)
    
    # izris
    ax1_s1_line.set_data(t, s1)
    ax1_title.set_text(f's1 = {f} Hz, amplituda {A:.2f}, faza {faza:.2f} pi')
    
    ax2_sk_prod_line.set_data([0, cos_s1], [0, sin_s1])
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(frekvence_list.shape[0]),
                              interval=1000,
                              blit=False)

In [ ]:
HTML(anim.to_jshtml())

## Skalarni produkt ob frekvenčnem neujemanju
### Spektralno prepuščanje (spectral leakage)
#### Več o tem v spodnjih zgledih - razlaga na voljo tudi na: https://en.wikipedia.org/wiki/Spectral_leakage

In [ ]:
%%capture
Fvz = 512 # frekvenca vzorcenja
T = 2 # dolzina signala v sekundah
i = np.arange(0, T*Fvz) / Fvz # casovni trenutki
f1 = 5  # frkevenca prve sinusoide v Hz
faza1 = 0
f2 = 5 # frekvenca druge sinusoide v Hz
faza2 = 0

s1 = np.sin(2 * np.pi * f1 * i + np.pi * faza1)
s2 = np.sin(2 * np.pi * f2 * i + np.pi * faza2)

f2_list = np.linspace(3, 7, 100)
skalarni_produkt = np.zeros(f2_list.shape[0])

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.plot(i, s1)
plt.title(f'{f1} Hz')
s2_line, = plt.plot(i, s2)
ax1_title = plt.title(f'{f2} Hz')


ax2 = plt.subplot(2, 1, 2)
plt.xlabel('Frekvenca neujemanja (Hz)')
sk_prod_line, = plt.plot(f2_list, skalarni_produkt)
sk_prod_dot,  = plt.plot(f2_list[0], skalarni_produkt[0], 'or')
plt.tight_layout()
    


def animate(n):
    # pripravimo nov s2 in pomnozimo z s1
    s2 = np.sin(2 * np.pi * (f2_list[n]) * i + faza2 * np.pi)
    skalarni_produkt[n] = s1.dot(s2)
    
    # izris
    s2_line.set_data(i, s2)
    ax1_title.set_text(f's1={f1} Hz, s2={f2_list[n]:.2f} Hz')
    
    sk_prod_line.set_data(f2_list, skalarni_produkt)
    sk_prod_dot.set_data(f2_list[n], skalarni_produkt[n])
    
    ax2.set_ylim(skalarni_produkt.min(), skalarni_produkt.max())
    
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(f2_list.shape[0]),
                              interval=50,
                              blit=False,
                              repeat=False)
    

In [ ]:
HTML(anim.to_jshtml())

## Analiza z FFT

Isti izračuni kot prejšnji primeri, ampak sedaj z uporabo hitre Fourierove transformacije.

In [ ]:
%%capture

Fvz = 512 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
n = np.arange(0, T*Fvz) # casovni trenutki

N = int(Fvz*T)
t = n/Fvz

f = 3 # frkevenca sinusoide v Hz
A = 2 # amplituda
faza = 0.25

s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal

# vzemimo 3 vzorec iz FFT transformacije 
S1 = np.fft.fft(s1)
sin_s1 = np.real(S1[3])
cos_s1 = np.imag(S1[3])


fig = plt.figure()
plt.subplot(2, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
ax1_s1_line, = plt.plot(t, s1)
ax1_title = plt.title(f'{f1} Hz, faza = {0} pi')

ax2 = plt.subplot(2, 1, 2)
ax2_sk_prod_line, = plt.plot([0, sin_s1], [0, cos_s1])
ax2.set_xlim(-600, 600)
ax2.set_ylim(-600, 600)
ax2.set_aspect('equal')
plt.xlabel('x os')
plt.ylabel('y os')
plt.tight_layout()

faze_list = np.arange(-1, 1, 0.1)
def animate(i):
    # pripravimo nas signal
    faza = faze_list[i]
    s1 = A * np.sin(2 * np.pi * f * n/Fvz + faza * np.pi) # signal
    
    # izracun z FFT
    S1 = np.fft.fft(s1)
    sin_s1 = np.real(S1[3])
    cos_s1 = np.imag(S1[3])

    
    # izris
    ax1_s1_line.set_data(t, s1)
    ax1_title.set_text(f's1 = {f} Hz, amplituda {A:.2f}, faza {faza:.2f} pi')
    
    ax2_sk_prod_line.set_data([0, sin_s1], [0, cos_s1])
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(faze_list.shape[0]),
                              interval=50,
                              blit=False)

In [ ]:
HTML(anim.to_jshtml())

## Polna analiza z baznimi funkcijami

In [ ]:
%%capture
Fvz = 20  # vzorcevalna frekvenca
T = 2     # dolzina signala v sekundah
N = Fvz*T # dolzina signala v vzorcih

n = np.arange(N) # indeksi posameznih vzorcev/meritev
t = n/Fvz        # casovni trenutki posameznih vzorcev/meritev

K = N//2+1

#s = np.random.randn(N)                     # nakljucen signal
s = (np.sin(2*np.pi*0.8*t + 2*np.pi*0.25)>0)      # kvadraten signal, osnovna frekvenca 0.8 Hz

S = np.zeros(s.shape, dtype=np.complex64) # tukaj bomo hranili rezultat analize

freq = np.arange(N)/N*Fvz

fig, ax = plt.subplots(3, 2)
ax=ax.ravel()

def label_prep(k):
    ax[0].set_title('signal')
    ax[0].set_xlabel('cas (s)')
    ax[0].set_ylabel('amplituda')

    ax[1].set_title(f'bazne funkcije {k}/{N}')
    ax[1].set_xlabel('cas (s)')
    ax[1].set_ylabel('amplituda')

    ax[2].set_title('Signal v frekv. prostoru')
    ax[2].set_xlabel('frekvenca (Hz)')
    ax[2].set_ylabel('vrednost komponente')


    ax[3].set_title('Amplitude')
    ax[3].set_xlabel('frekvenca (Hz)')
    ax[3].set_ylabel('amplituda (abs)')

    ax[4].set_title('Faze')
    ax[4].set_xlabel('frekvenca (Hz)')
    ax[4].set_ylabel('faza')
    
    ax[5].set_axis_off()
label_prep(0)
plt.tight_layout()

def animate(k):
    if k==0:
        S[:] = 0
    # tukaj je koda analiza za vsak k posebej
    sin_base = -1*np.sin(2 * np.pi  * n*k/N) # sinusoida
    cos_base = np.cos(2 * np.pi  * n*k/N) # kosinusoida
    
    S[k] = cos_base.dot(s) + 1j*sin_base.dot(s)
    
    # izris
    for a in ax:
        a.cla()
    
    ax[0].plot(t, s)
    ax[1].plot(t, sin_base)
    ax[1].plot(t, cos_base)
    
    ax[2].plot(freq, np.real(S))
    ax[2].plot(freq, np.imag(S))
    
    P = np.arctan2(np.imag(S), np.real(S)) # dobimo faze

    ax[3].plot(freq, np.abs(S))    
    ax[4].plot(freq, P)
    
    label_prep(k)

anim = animation.FuncAnimation(fig,
                              animate,
                              frames=np.arange(K),
                              interval=100,
                              blit=False,
                              repeat=False)


In [ ]:
HTML(anim.to_jshtml())

### Rekonstrukcija z baznimi funkcijami

In [ ]:
%%capture
s_rec = np.zeros(s.shape, dtype=np.complex128)

fig, ax = plt.subplots(3, 1)
ax=ax.ravel()

def animate(k):
    # na prvem koraku ga nastavimo na 0
    if k==0:
        s_rec[:] = 0
        
    # tukaj je koda analiza za vsak k posebej
    sin_base = -1*np.sin(2 * np.pi  * n*k/N) # sinusoida
    cos_base = np.cos(2 * np.pi  * n*k/N) # kosinusoida
    
    a = np.real(S[k])
    b = np.imag(S[k])

    # pri rekonstrukciji je potrebna se normalizacija 
    # ker nase bazne funkcije niso bili vektorji dolzine 0
    if k == 0:
        norm_fact = S.shape[0]
    else:
        norm_fact = S.shape[0]/2
    
    a/=norm_fact
    b/=norm_fact
    
    s_rec[:] += a*cos_base + b*sin_base
    
    # to lahko zamenjamo z eulerjevo formulo, to je razlog da uporabljamo kompleksna stevila
    # https://en.wikipedia.org/wiki/Euler%27s_formula
    # primer je potrebno nekoliko prilagoditi
    
    #s_rec[:] += S[k]/S.shape[0]*np.exp(1j*2*np.pi*n*k/N)
    
    # izris
    for v in ax:
        v.cla()
    
    ax[0].plot(t, s)
    ax[0].plot(t, np.real(s_rec))
    ax[0].plot(t, np.imag(s_rec))
    
    ax[1].plot(t, a*cos_base)
    ax[1].plot(t, b*sin_base)
    
    ax[2].plot(freq, np.real(S))
    ax[2].plot(freq, np.imag(S))
    ax[2].plot(freq[k], np.real(S[k]),'ro')
    ax[2].plot(freq[k], np.imag(S[k]),'ro')
    
    #label_prep(k)

    

anim = animation.FuncAnimation(fig,
                              animate,
                              frames=np.arange(K),
                              interval=1000,
                              blit=False,
                              repeat=False)

In [ ]:
HTML(anim.to_jshtml())

## Rekonstrukcija z amplitudo in fazo

In [ ]:
%%capture
s_rec = np.zeros(s.shape)

S_amplitude = np.abs(S)
S_faze = np.arctan2(np.imag(S), np.real(S)) 
    
fig, ax = plt.subplots(4, 1)
ax=ax.ravel()

def animate(k):
    # na prvem koraku ga nastavimo na 0
    if k==0:
        s_rec[:] = 0
        
    # pri rekonstrukciji je potrebna se normalizacija 
    # ker nase bazne funkcije niso bili vektorji dolzine 0
    if k == 0:
        norm_fact = S.shape[0]
    else:
        norm_fact = S.shape[0]//2
    
    # dobimo amplitudo in fazo
    A = S_amplitude[k]/norm_fact
    P = S_faze[k]
    
    # pripravimo sinusoido, oziroma kosinusoido
    cos_add = A * np.cos(2 * np.pi  * n*k/N + P)   

    s_rec[:] += cos_add
    
    # izris
    for v in ax:
        v.cla()
    
    ax[0].plot(t, s)
    ax[0].plot(t, s_rec)
    
    ax[1].plot(t, cos_add)
    
    ax[2].plot(freq, S_amplitude)
    ax[2].plot(freq[k], S_amplitude[k],'ro')

    ax[3].plot(freq, S_faze)
    ax[3].plot(freq[k], S_faze[k],'ro')
    
    #label_prep(k)

    

anim = animation.FuncAnimation(fig,
                              animate,
                              frames=K,
                              interval=1000,
                              blit=False,
                              repeat=False)

In [ ]:
HTML(anim.to_jshtml())

## Analiza z FFT - pogled z druge perspektive

### Spreminjanje faze

In [ ]:
%%capture

Fvz = 64 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
i = np.arange(0, T*Fvz) / Fvz # casovni trenutki
f1 = 5 # frkevenca prve sinusoide v Hz

A = 1 # amplituda

faza1 = 0

s1 = A * np.sin(2 * np.pi * f1 * i + faza1 * np.pi)

X = np.fft.fft(s1) # FFT

faze_list = np.arange(-1, 1, 0.1)

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
ax1_s1_line, = plt.plot(i, s1)
ax1_title = plt.title(f's1 = {f1} Hz, faza {faza1:.2f} pi')


ax2 = plt.subplot(3, 1, 2)
ax2_xlabel = plt.xlabel('frekvenca (Hz)')
ax2_ylabel = plt.ylabel('amplituda')
ax2_markerline, ax2_stemlines, ax2_baselin, = ax2.stem(i/T*Fvz, X.real)


ax3 = plt.subplot(3, 1, 3)
ax3_xlabel = plt.xlabel('frekvenca (Hz)')
ax3_ylabel = plt.ylabel('amplituda')
ax3_markerline, ax3_stemlines, ax3_baselin, = ax3.stem(i/T*Fvz, X.imag)

plt.tight_layout()
    
    
def animate(n):
    faza1 = faze_list[n]
    s1 = A * np.sin(2 * np.pi * f1 * i + faza1 * np.pi)
    
    X = np.fft.fft(s1) # FFT
    
    ax1_s1_line.set_data(i, s1)
    ax1_title.set_text(f's1 = {f1} Hz, faza {faza1:.2f} $\pi$')
    
    ax2.clear()
    ax2_markerline, ax2_stemlines, ax2_baselin, = ax2.stem(i/T*Fvz, X.real)
    ax2.set_ylim((-32, 32))
    ax2.set_title('realni del fft - amplituda kosinusov')
    ax2_xlabel.set_text('frekvenca (Hz)')
    ax2_ylabel.set_text('amplituda')
    
    ax3.clear()
    ax3_markerline, ax3_stemlines, ax3_baselin, = ax3.stem(i/T*Fvz, X.imag)
    ax3.set_ylim((-32, 32))
    ax3.set_title('imaginarni del fft - amplituda sinusov')
    ax3_xlabel.set_text('frekvenca (Hz)')
    ax3_ylabel.set_text('amplituda')
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(faze_list.shape[0]),
                              interval=50,
                              blit=False)

In [ ]:
HTML(anim.to_jshtml())

### Spreminjanje frekvence 

In [ ]:
%%capture

Fvz = 64 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
i = np.arange(0, T*Fvz) / Fvz # casovni trenutki
f1 = 5 # frkevenca prve sinusoide v Hz

A = 1 # amplituda

faza1 = 0.25

s1 = A * np.sin(2 * np.pi * f1 * i + faza1 * np.pi)

X = np.fft.fft(s1) # FFT

frekvence_list = np.arange(0, 5, 0.10)

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.xlabel('čas (s)')
plt.ylabel('amplituda')
ax1_s1_line, = plt.plot(i, s1)
ax1_title = plt.title(f's1 = {f1} Hz, faza {faza1:.2f} pi')


ax2 = plt.subplot(3, 1, 2)
ax2_xlabel = plt.xlabel('frekvenca (Hz)')
ax2_ylabel = plt.ylabel('amplituda')
ax2_markerline, ax2_stemlines, ax2_baselin, = ax2.stem(i/T*Fvz, X.real)


ax3 = plt.subplot(3, 1, 3)
ax3_xlabel = plt.xlabel('frekvenca (Hz)')
ax3_ylabel = plt.ylabel('amplituda')
ax3_markerline, ax3_stemlines, ax3_baselin, = ax3.stem(i/T*Fvz, X.imag)

plt.tight_layout()
    
    
def animate(n):
    f1 = frekvence_list[n]
    s1 = A * np.sin(2 * np.pi * f1 * i + faza1 * np.pi)
    
    X = np.fft.fft(s1) # FFT
    
    ax1_s1_line.set_data(i, s1)
    ax1_title.set_text(f's1 = {f1:.2f} Hz, faza {faza1:.2f} pi')
    
    ax2.clear()
    ax2_markerline, ax2_stemlines, ax2_baselin, = ax2.stem(i/T*Fvz, X.real)
    ax2.set_ylim((-32, 32))
    ax2.set_title('realni del fft - amplituda kosinusov')
    ax2_xlabel.set_text('frekvenca (Hz)')
    ax2_ylabel.set_text('amplituda')
    
    ax3.clear()
    ax3_markerline, ax3_stemlines, ax3_baselin, = ax3.stem(i/T*Fvz, X.imag)
    ax3.set_ylim((-32, 32))
    ax3.set_title('imaginarni del fft - amplituda sinusov')
    ax3_xlabel.set_text('frekvenca (Hz)')
    ax3_ylabel.set_text('amplituda')
    
anim = animation.FuncAnimation(fig,
                              animate,
                              frames=range(frekvence_list.shape[0]),
                              interval=50,
                              blit=False)

In [ ]:
HTML(anim.to_jshtml())

## Povečanje ločljivosti

In [ ]:
Fvz = 64 # frekvenca vzorcenja
T = 1 # dolzina signala v sekundah
i = np.arange(0, T*Fvz) / Fvz # casovni trenutki
f1 = 5.25 # frkevenca prve sinusoide v Hz

A = 1 # amplituda

faza1 = 0.25

s1 = A * np.sin(2 * np.pi * f1 * i + faza1 * np.pi)

# izracunamo FFT, povecamo izhod za faktor 10

S1 = np.fft.fft(s1, s1.shape[0]*10)

# tako lahko dobimo tudi seznam frekvenc
freq_list = np.fft.fftfreq(S1.shape[0], 1/Fvz)
#freq_list = np.arange(S1.shape[0])/S1.shape[0]*Fvz

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(i, s1)
plt.title('signal')
plt.xlabel('cas (s)')
plt.ylabel('amplituda')
plt.subplot(2, 1, 2)
plt.stem(freq_list, np.abs(S1))
plt.title('kosinusoide')
plt.ylabel('amplituda')
plt.xlabel('frekvence')
plt.tight_layout()

## Lastnosti DFT

### Linearnost DFT-ja

In [ ]:
x = np.random.randn(128, 1) # ustvarim signal x (ter ga zamaknemo okoli osi)
X = np.fft.fft(x, axis=0)   # in njegovo frekvencno transformiranko
y = np.random.randn(128, 1) # ustvarim signal y (ter ga zamaknemo okoli osi)
Y = np.fft.fft(y, axis=0)   # in njegovo frekvencno transformiranko

a = np.random.rand() # ustvarimo nakljucne mnozilne faktorje
b = np.random.rand() # ustvarimo nakljucne mnozilne faktorje

z_1 = a*x + b*y             # signal z je linearna kombinacija signalov x in y
Z_1 = np.fft.fft(z_1, axis=0) # izracunamo DFT signala z

Z_2 = a*X + b*Y


# graficni preizkus linearnosti DFT-ja
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(np.real(Z_1), linewidth=2)
plt.plot(np.real(Z_2), 'r', linewidth=1)
plt.xlabel('frekvenca')
plt.ylabel('realni del')
plt.title('fft(a*x + b*y) in a*fft(x) + b*fft(y): primerjava realnih delov')
plt.subplot(2, 1, 2)
plt.plot(np.imag(Z_1),linewidth=2);
plt.plot(np.imag(Z_2),'r', linewidth=1);
plt.ylabel('imaginarni del')
plt.xlabel('frekvenca')
plt.title('fft(a*x + b*y) in a*fft(x) + b*fft(y): primerjava imaginarnih delov')
plt.subplots_adjust(hspace=0.7)
plt.show()
# nekaj nekaj zaokrozevanje

### Parsevalov teorem

In [ ]:
x = np.random.rand(128, 1) # ustvarim signal 
X = np.fft.fft(x, axis=0)  # in njegovo frekvencno transformiranko 

print('Energija v t domeni:', np.matmul(x.T, x)[0][0])
print('Energija v f domeni:', np.mean(np.power(np.abs(X),2)))

### Konvolucija in frekvenčni prostor

Konvolucija v frekvenčnem prostoru se izračuna kot množenje istoležnih elementov v frekvenčnem prostoru.

Velja tudi obratno, konvolucija v frekvenčnem prostoru je množenje istoležnih elementov v časovnem prostoru.

#### Konvolucija v časovnem prostoru == množenje v frekvenčnem

Ta je preprosta in praktično uporabna.


In [ ]:
x = np.array([1,2,1])
h = np.array([1, -1])

y = np.convolve(x, h)

# tako x kot h je tukaj potrebno podaljšati, da bomo lahko množili istoležne elemente
# dolžina je enaka kot je bila pri preprosti konvoluciji
X = np.fft.fft(x, n=x.shape[0]+h.shape[0]-1)
H = np.fft.fft(h, n=x.shape[0]+h.shape[0]-1)

Y = X*H
# inverzna fourierova transformacija rekonstruira originalni signal 
# pri tem je običajno potrebno vzeti samo realni del (tudi ta funkcija vrne kompleksna števila)
y2 = np.real(np.fft.ifft(Y))

y, y2

#### Konvolucija v frekvečnem prostoru == množenje v časovnem

Zaradi specifik hitre Fourierove transformacije je potrebnih nekaj nenavadnih računskih trikov, da dobimo identičen rezultat.

V praksi tudi ne bi izvajali koknvolucije v frekvenčnem prostoru, je pa ta lastnost pomembna za razumevanje vpliva oken, ki jih bomo uporabljali kasneje.


In [ ]:
x = np.array([1,2,1,2])
h = np.array([-1, 1, -1, 0])

y = x*h

# tako x kot h je tukaj potrebno podaljšati, da bomo lahko množili istoležne elemente
# dolžina je enaka kot je bila pri preprosti konvoluciji
X = np.fft.fft(x)
H = np.fft.fft(h)

# konvoluciji ne smem dovoliti da obda naš X z 0, kjer bo H šrtlel preko roba
# funkcija se mora ob robovi ponoviti, zato jo tu trikrat ponovim
X3 = np.concatenate((X,X,X)) 

# naš impulzni odziv v tem prostoru ojača vrednosti, sorazmerno z številom vrednosti
# zato dodamo še deljenje
Y = np.convolve(X3, H, mode='full')/H.shape[0]
Y=Y[X.shape[0]:][:X.shape[0]]

# inverzna fourierova transformacija rekonstruira originalni signal 
# pri tem je običajno potrebno vzeti samo realni del (tudi ta funkcija vrne kompleksna števila)
y2 = np.real(np.fft.ifft(Y, n=x.shape[0]))

y, y2

## Analiza signala

### Analiza sintetičnega signala

In [ ]:
Fvz = 512 # vzorčevalna frekvenca

T = 2 # dolžina signala v sekundah
t = np.arange(0, T, 1.0/Fvz) # časovni vektor

f1 = 5; A1 = 7; faza1 = 1.0;  # parametri prve sinusuide
f2 = 35; A2 = 3; faza2 = 0.1; # parametri druge sinusuide

s1 = A1 * np.sin(2 * np.pi * f1 * t + 2*np.pi*faza1)
s2 = A2 * np.sin(2 * np.pi * f2 * t + 2*np.pi*faza2)

# ustvarimo kompleksni signal
x = s1 + s2


plt.figure()
plt.plot(s1)
plt.xlabel('Vzorec'); plt.ylabel('Amplituda');
plt.show()

plt.figure()
plt.plot(s2)
plt.xlabel('Vzorec'); plt.ylabel('Amplituda');
plt.show()

plt.figure()
plt.plot(x)
plt.xlabel('Vzorec'); plt.ylabel('Amplituda');
plt.show()

In [ ]:
X = np.fft.fft(x) # FFT!!

M = np.abs(X) # dobimo amplitude
P = np.arctan2(np.imag(X), np.real(X)) # dobimo faze
P1 = P * ((M > 0.1) * 1) # samo izbrane faze

M_norm = M/(X.shape[0]/2)
plt.figure()
plt.subplot(2, 2, 1)
plt.stem(t/T*Fvz, np.real(X)) # narišemo amplitude kosinusov
plt.xlabel('Frekvenca'); plt.ylabel('Amplituda');
plt.title('Amplituda kosinusov')


plt.subplot(2, 2, 2)
plt.stem(t/T*Fvz, np.imag(X) * -1.0) # narišemo amplitude sinusov
plt.xlabel('Frekvenca'); plt.ylabel('Amplituda');
plt.title('Amplituda sinusov')

plt.subplot(2, 2, 3)
plt.stem(t/T*Fvz, M_norm) # Narišemo amplitude. Na X osi so frekvence!!
plt.xlabel('Frekvenca'); plt.ylabel('Amplituda');
plt.title('Skupna aplituda prisotnih sinusoid')

plt.subplot(2, 2, 4)
plt.stem(t/T*Fvz, P)
plt.title('Faze vseh prisotnih sinusoid')

plt.subplot(2, 2, 4)
plt.stem(t/T*Fvz, P1, markerfmt='ro') # Narišemo faze
plt.xlabel('Frekvenca'); plt.ylabel('Amplituda');
plt.title('Faze samo prisotnih sinusoid')
plt.tight_layout()
plt.show()

### Analiza realnega signala

In [ ]:
import sounddevice as sd
from scipy.io import wavfile

In [ ]:

#Fvz, h = wavfile.read('Sinus.wav')
#Fvz, h = wavfile.read('Zvizganje1.wav')
#Fvz, h = wavfile.read('Zvizganje2.wav')
Fvz=44100
time=5.0
h = sd.rec(int(Fvz*time), Fvz, channels=1, blocking=True)

print(Fvz, np.size(h))

#h = h / np.max(np.abs(h)) # normalizirajmo

plt.figure()
plt.plot(np.arange(0,h.shape[0])/Fvz, h)
plt.xlabel('čas (s)'); plt.ylabel('amplituda');
plt.show()

In [ ]:
sd.play(h, Fvz)

In [ ]:
sd.stop()

In [ ]:
H = np.fft.fft(h, axis=0) # FFT

plt.figure()
plt.stem(np.abs(H[1:-1]))
plt.xlabel('frekvenca'); plt.ylabel('amplituda');
plt.show()

plt.figure()
plt.plot((np.arange(0, np.size(h))/np.size(h)*Fvz)[1:-1], np.abs(H[1:-1]))
plt.xlabel('frekvenca'); plt.ylabel('amplituda');
plt.show()